<a href="https://colab.research.google.com/github/JorgeZarama/TFM-final-models/blob/main/Dashboard_An%C3%A1lisis_de_Sentimiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importar Librerias

In [ ]:
import numpy as np
import pandas as pd
import string
import re
import unidecode
import codecs
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, Dropout, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import warnings
from tensorflow.keras.models import load_model
warnings.filterwarnings('ignore')

## Funciones a Utilizar

## Buscamos Mensajes de REDDIT

In [ ]:
import praw
import pandas as pd
from datetime import datetime, timedelta
import time

# Configura tu cliente PRAW con las credenciales de Reddit
reddit = praw.Reddit(
    client_id="uXHojHc5dggM8xUWPu8BUw",
    client_secret="cTLJhrNVBIzdlDO9fKj74_NX_m0BKw",
    password="P2uGZ6UzShF%Te?",
    user_agent='Vegetable-Truth-5760/Análisis-de-Sentimiento-WallStreetBets/v1.0',
    username="Vegetable-Truth-5760"
)

def fetch_recent_comments(query, subreddits, days=3):
    """
    Busca comentarios recientes en Reddit basados en una palabra clave.

    Parámetros:
        query (str): Palabra clave o símbolo de la acción (ejemplo: "GOOGL").
        subreddits (str): Lista de subreddits separados por "+" (ejemplo: "investing+stocks+wallstreetbets").
        days (int): Número de días hacia atrás para buscar comentarios. Por defecto, 3 días.

    Retorna:
        DataFrame: Un DataFrame con los comentarios organizados de más reciente a más antiguo.
    """
    comments_data = []  # Lista para almacenar los datos

    # Calcular la fecha de corte para buscar comentarios
    end_time = datetime.utcnow()
    start_time = end_time - timedelta(days=days)
    start_epoch = int(start_time.timestamp())

    try:
        # Buscar publicaciones en los subreddits especificados
        search_results = reddit.subreddit(subreddits).search(
            query=query,
            sort='new',
            syntax='lucene',
            time_filter='all'  # Filtrar todos los tiempos, luego ajustamos por la fecha
        )

        for submission in search_results:
            if submission.created_utc < start_epoch:
                continue  # Saltar publicaciones fuera del rango de días

            submission.comments.replace_more(limit=0)
            for comment in submission.comments.list():
                if comment.created_utc >= start_epoch:
                    comments_data.append({
                        'comment_id': comment.id,
                        'parent_id': comment.parent_id,
                        'submission_id': comment.link_id,
                        'created_utc': datetime.utcfromtimestamp(comment.created_utc),
                        'body': comment.body,
                        'submission_title': submission.title,
                        'subreddit': comment.subreddit.display_name,
                        'author': comment.author.name if comment.author else 'Deleted'
                    })

        # Crear un DataFrame y ordenarlo por fecha
        df = pd.DataFrame(comments_data)
        if not df.empty:
            df.sort_values(by='created_utc', ascending=False, inplace=True)

        return df

    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()  # Retorna un DataFrame vacío en caso de error


Version 7.7.1 of praw is outdated. Version 7.8.1 was released Friday October 25, 2024.


## Preprocesar Datos

In [ ]:
def preprocess_tweet(text):
    text = re.sub(r'http\S+|www.\S+', '', text, flags=re.MULTILINE)
    contraction_mapping = {"ain't": "is not", "aren't": "are not", "can't": "cannot",
                           "can't've": "cannot have", "'cause": "because", "could've": "could have",
                           "couldn't": "could not", "couldn't've": "could not have", "didn't": "did not",
                           "doesn't": "does not", "don't": "do not", "hadn't": "had not",
                           "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would", "he'd've": "he would have", "he'll": "he will",
                           "he'll've": "he will have", "he's": "he is", "how'd": "how did",
                           "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will",
                           "I'll've": "I will have", "I'm": "I am", "I've": "I have",
                           "i'd": "i would", "i'd've": "i would have", "i'll": "i will",
                           "i'll've": "i will have", "i'm": "i am", "i've": "i have",
                           "isn't": "is not", "it'd": "it would", "it'd've": "it would have",
                           "it'll": "it will", "it'll've": "it will have", "it's": "it is",
                           "let's": "let us", "ma'am": "madam", "mayn't": "may not",
                           "might've": "might have", "mightn't": "might not", "mightn't've": "might not have",
                           "must've": "must have", "mustn't": "must not", "mustn't've": "must not have",
                           "needn't": "need not", "needn't've": "need not have", "o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                           "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would",
                           "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have",
                           "she's": "she is", "should've": "should have", "shouldn't": "should not",
                           "shouldn't've": "should not have", "so've": "so have", "so's": "so as",
                           "this's": "this is",
                           "that'd": "that would", "that'd've": "that would have", "that's": "that is",
                           "there'd": "there would", "there'd've": "there would have", "there's": "there is",
                           "here's": "here is",
                           "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
                           "they'll've": "they will have", "they're": "they are", "they've": "they have",
                           "to've": "to have", "wasn't": "was not", "we'd": "we would",
                           "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                           "we're": "we are", "we've": "we have", "weren't": "were not",
                           "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is",
                           "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have",
                           "who's": "who is", "who've": "who have", "why's": "why is",
                           "why've": "why have", "will've": "will have", "won't": "will not",
                           "won't've": "will not have", "would've": "would have", "wouldn't": "would not",
                           "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                           "y'all'd've": "you all would have", "y'all're": "you all are", "y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will",
                           "you'll've": "you will have", "you're": "you are", "you've": "you have"}

    apostrophe_handled = re.sub("’", "'", text)                                                                         # keep only one type of apostrophe
    try:
        decoded = unidecode.unidecode(codecs.decode(apostrophe_handled, 'unicode_escape'))                              # decode in ascii
    except:
        decoded = unidecode.unidecode(apostrophe_handled)
    expanded = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in decoded.split(" ")])        # map contraction form into expanded form
    noticks = re.sub(r'\$\S+', '', expanded, flags=re.MULTILINE)                                                        # remove tickers
    nousers = re.sub(r'@\S+', '', noticks, flags=re.MULTILINE)
    punctuationfree="".join([i.lower() for i in nousers if i not in string.punctuation])
    tokens = re.split('\W+', punctuationfree)
    joined = ' '.join(tokens).lower()
    spell_corrected = re.sub(r'(.)\1+', r'\1\1', joined)
    return spell_corrected

In [ ]:
def Preprocesar_Datos(df_new):
    df_new['created_utc'] = pd.to_datetime(df_new['created_utc'])
    df_new = df_new[["created_utc", "body", "submission_title"]]
    df_new = df_new.dropna(subset=['body'])
    df_new['body'] = df_new['body'].astype(str)
    df_new['Preprocessed_text'] = df_new['body'].apply(preprocess_tweet)
    df_new = df_new[df_new['Preprocessed_text'].str.strip().astype(bool)]
    df_new = df_new.dropna(subset=['Preprocessed_text'])
    return df_new

## Predecir Datos

In [ ]:
#from keras.models import load_model
from tensorflow.keras.models import load_model
import pickle

model = load_model('AnalisisSentimientoFinal.keras')
# Cargar el tokenizador desde el archivo
with open('tokenizer.pkl', 'rb') as file:
    tokenizer = pickle.load(file)

def Predecir_Datos(df_new):
    x_new_seq = tokenizer.texts_to_sequences(df_new['Preprocessed_text'])
    x_new_seq = pad_sequences(x_new_seq, maxlen=20, padding='post')
    predictions = model.predict(x_new_seq)
    df_new['predictions'] = predictions
    return df_new.reset_index(drop=True)



## Graficar Dashboard

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

def crear_dashboard_termometro(avg_prediction, query):
    """
    Crea un dashboard con un gráfico de termómetro y leyendas.

    Args:
        avg_prediction (float): Promedio de las predicciones.
        query (str): Palabra clave o símbolo de la acción.
    """

    fig = go.Figure(go.Indicator(
        mode="gauge+number",
        value=avg_prediction,
        domain={'x': [0, 1], 'y': [0, 1]},
        gauge={'axis': {'range': [0, 1]},
                'bar': {'color': "darkblue" if avg_prediction < 0.5 else "green"},
                'steps' : [
                    {'range': [0, 0.5], 'color': "lightgray"},
                    {'range': [0.5, 1], 'color': "lightgreen"}],
                'threshold' : {
                    'line': {'color': "red", 'width': 4},
                    'thickness': 0.75,
                    'value': 0.5}}))

    fig.update_layout(
        title=f"Sentimiento hacia {query}",
        height=400,
        annotations=[
            dict(text="Bullish", x=0.85, y=0.8, font=dict(size=16)),
            dict(text="Bearish", x=0.15, y=0.8, font=dict(size=16))
        ]
    )

    fig.show()




def crear_dashboard_comentarios(df, n=10):
    """
    Crea un dashboard con los comentarios más recientes.

    Args:
        df (pandas.DataFrame): DataFrame con los datos procesados.
        n (int): Número de comentarios a mostrar.
    """

    df_recent = df.sort_values('created_utc', ascending=False).head(n)
    fig = px.bar(df_recent, x='body', y='predictions', orientation='h')
    fig.update_layout(
        title="Comentarios más recientes",
        xaxis_title="Predicción",
        yaxis_title="Comentario",
        height=500
    )
    fig.update_xaxes(visible=False)

    fig.show()



In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import random

# Lista de palabras clave
palabras_clave = ["TSM", "ASML", "GOOGL", "LRCX"]
subreddits = "investing+stocks+wallstreetbets"
days = 2

def crear_dashboards(palabras_clave, subreddits, days):
    for query in palabras_clave:
        df_comments = fetch_recent_comments(query, subreddits, days)

        if not df_comments.empty:
            # Preprocesamiento y predicción (asumiendo que tienes estas funciones definidas)
            df = Preprocesar_Datos(df_comments)
            df = Predecir_Datos(df)

            # Calcular el promedio de predicciones y seleccionar comentarios aleatorios
            avg_prediction = df['predictions'].mean()
            random_comments = df['body'].sample(n=5).tolist()

            # Crear los dashboards
            crear_dashboard_termometro(avg_prediction, query)
            crear_dashboard_comentarios(df)
        else:
            print(f"No se encontraron comentarios recientes para {query}")

# Llamar a la función principal
crear_dashboards(palabras_clave, subreddits, days)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


No se encontraron comentarios recientes para ASML
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


No se encontraron comentarios recientes para LRCX
